# Computing and creating the flights_aircrafts table and the optimal flights table

In [1]:
import sys
# This variable should indicate the path from this Jupyter Notebook to the root directory of the repo.
root_path = '../'
# Adds the repo's root to the list of paths
sys.path.append(root_path)

# Package to read yml files
import yaml
# Package to handle file paths
import os
# Package to deal with DataFrames
import pandas as pd
# Package to plot stuff
import matplotlib.pyplot as plt
# Package for numerical and array handling
import numpy as np
# Package to read and write to .sqlite files
import sqlite3
# Package to keep track of time
import datetime
# Package to get flight aircraft info
from traffic.core import Flight

# Function to clear output from jupyter notebook
from IPython.display import clear_output
# Package for compressing dataframes into file
from src.data import compressors
# Package for defining and fitting weather models
from src.models import weather
# Utilities package
from src.common import utils
# Package for interpolating and estimating weather
from src.analysis import weather_interpolator

def sigmoid(x, mu = 0, sig = 1):
    return 1/(1+np.exp(-(x - mu)/sig))

# Time Integral
def integrate_time(state_vectors):
    return state_vectors['time'].iloc[-1] - state_vectors['time'].iloc[0]

def integrate_fuel(state_vectors):
    return state_vectors['used_fuel'].iloc[-1]

# Wind Integral
def integrate_wind(state_vectors):
    speed_of_sound = 666.739
    return np.sum(state_vectors['sknt'])/(speed_of_sound*len(state_vectors))

# Air Density Integral
def integrate_air_density(state_vectors):
    sea_level_density = 1.204
    return np.sum(state_vectors['air_density'])/(sea_level_density*len(state_vectors))

def integrate_air_pressure(state_vectors):
    sea_level_pressure = 1013.25
    return np.sum(state_vectors['air_pressure'])/(sea_level_pressure*len(state_vectors))

def integrate_clouds(state_vectors):
    max_clouds = 1
    return np.sum(state_vectors['clouds'])/(max_clouds*len(state_vectors))

def integrate_severity(state_vectors):
    max_severity = 1
    return np.sum(state_vectors['severity'])/(max_severity*len(state_vectors))


def integrate_distance(state_vectors):
    d = 0
    for row_a, row_b in zip(state_vectors[:-1].itertuples(), state_vectors[1:].itertuples()):
        d += utils.haversine_distance(row_a.lat, row_a.lon, row_b.lat, row_b.lon)
    return d

# Path from this notebook to the root directory
root_path = os.path.normpath(root_path)
# Path from root to the desired config file
config_path_from_root = os.path.normpath('config/config.yml')
# Defining path from this notebook to config file
config_path = os.path.join(root_path, config_path_from_root)

# Loading config file
with open(config_path, 'r',  encoding='utf8') as file:
    config = yaml.safe_load(file)

# Defining "clear-output" function to feed into logger
def clear():
    clear_output(wait=True)

# Creates an instance of a logger class to log all that happens, optional (but encouraged).
logger = utils.Logger(config, clear_function=None)

In [2]:
flights_database = '../data/flight/KLAX_KSFO_2023-01-01_2023-01-31.sqlite'

conn = sqlite3.connect(flights_database)

cursor = conn.cursor()

flights = pd.read_sql_query(f"""
    SELECT *
    FROM flights
    ;""",conn)

aicraft_db = {'icao24':[], 'registration':[], 'typecode':[]}
for icao24 in flights['icao24']:
    clear_output(wait=True)
    print(icao24)
    df = pd.DataFrame({'icao24': [icao24], 'timestamp': [0]})
    ff = Flight(df)
    try:
        ff.aircraft.get('icao24', icao24)
        ff.aircraft.get('registration', 'Other')
        ff.aircraft.get('typecode', 'Other')
        aicraft_db['icao24'].append(ff.aircraft.get('icao24', icao24))
        aicraft_db['registration'].append(ff.aircraft.get('registration', 'Other'))
        aicraft_db['typecode'].append(ff.aircraft.get('typecode', 'Other'))
    except:
        aicraft_db['icao24'].append(icao24)
        aicraft_db['registration'].append('Other')
        aicraft_db['typecode'].append('Other')

aicraft_db = pd.DataFrame(aicraft_db)

aicraft_db['icao24'] = aicraft_db['icao24'].fillna('Other')
aicraft_db['registration'] = aicraft_db['registration'].fillna('Other')
aicraft_db['typecode'] = aicraft_db['typecode'].fillna('Other')

cursor.execute("DROP TABLE IF EXISTS flights_aircraft;")

# Create the new table
create_table_query = f'''
    CREATE TABLE flights_aircraft (
        icao24 TEXT PRIMARY KEY,
        registration TEXT,
        typecode TEXT
    );
'''
# Create the new table if it doesn't exist
cursor.execute(create_table_query)

aicraft_db['typecode'] = aicraft_db['typecode'].apply(lambda x: 'Other' if x == '' else x)

for index, row in aicraft_db.iterrows():
    # Preparing the data to be inserted
    new_columns = ['icao24', 'registration', 'typecode']
    insert_data = tuple(row[col] for col in new_columns)

    # Creating query to insert new values
    insert_query = f'''
        INSERT INTO flights_aircraft ({', '.join(new_columns)})
        VALUES ({', '.join('?' * len(insert_data))})
        ON CONFLICT(icao24) DO UPDATE SET
        {', '.join([f"{col} = excluded.{col}" for col in new_columns])};
    '''
    print(insert_data)
    cursor.execute(insert_query, insert_data)
        
    conn.commit()
conn.close()
print("Done")

a36200
('a8c6e1', 'N665AW', 'A320')
('acdc5a', 'N928EV', 'CRJ2')
('a05410', 'N120DU', 'BCS1')
('aac3f4', 'N793JB', 'A320')
('a183d3', 'N197SY', 'E75L')
('aa4932', 'N762CK', 'B763')
('ab6247', 'N8321D', 'B738')
('a163c2', 'N189SY', 'E75L')
('71be10', 'HL7610', 'B748')
('71bc13', 'HL7413', 'B744')
('a837a6', 'N62895', 'B739')
('a1d93b', 'N218WN', 'B737')
('a517a1', 'N427WN', 'B737')
('a373da', 'N321US', 'A320')
('a666cd', 'N511SY', 'E75L')
('a4efe4', 'N417UA', 'A320')
('a4b5b6', 'N402SY', 'E75L')
('a44f0d', 'N377NW', 'A320')
('abd94d', 'N8625A', 'B738')
('a591d5', 'N458WN', 'B737')
('a8bc71', 'N662JB', 'A320')
('a6705c', 'N514AS', 'B738')
('a4493d', 'N37535', 'B39M')
('a4c492', 'N406SY', 'E170')
('a64305', 'N502SY', 'E75L')
('a2abda', 'N271SY', 'E75L')
('a4cc1c', 'N408UA', 'A320')
('71bc13', 'HL7413', 'B744')
('a1878a', 'N198SY', 'E75L')
('a591d5', 'N458WN', 'B737')
('a8bc71', 'N662JB', 'A320')
('a6705c', 'N514AS', 'B738')
('a4493d', 'N37535', 'B39M')
('a4c492', 'N406SY', 'E170')
('a6430

## Assigning optimal flight for each aircraft type

In [3]:
conn = sqlite3.connect(flights_database)

cursor = conn.cursor()

query = f"""
    SELECT fs.flight_id, fi.fuel, fa.typecode
    FROM flights AS fs
    JOIN flights_integrals AS fi ON fs.flight_id = fi.flight_id
    LEFT JOIN flights_aircraft AS fa ON fs.icao24 = fa.icao24
;"""

flights = pd.read_sql_query(query, conn)

unique_aircrafts = np.unique(flights['typecode'])

optimal_flights = {'flight_id':[], 'typecode':[]}

for aircraft in unique_aircrafts:
    mask = flights['typecode'] == aircraft
    optimal_id = flights[mask].sort_values('fuel')['flight_id'].iloc[0]
    optimal_flights['flight_id'] += [optimal_id]
    optimal_flights['typecode'] += [aircraft]
    
optimal_flights = pd.DataFrame(optimal_flights)

cursor.execute("DROP TABLE IF EXISTS optimal_flights;")

# Create the new table
create_table_query = f'''
    CREATE TABLE optimal_flights (
        flight_id TEXT PRIMARY KEY,
        typecode TEXT
    );
'''

cursor.execute(create_table_query)


for index, row in optimal_flights.iterrows():
    # Preparing the data to be inserted
    new_columns = ['flight_id', 'typecode']
    insert_data = tuple(row[col] for col in new_columns)

    # Creating query to insert new values
    insert_query = f'''
        INSERT INTO optimal_flights ({', '.join(new_columns)})
        VALUES ({', '.join('?' * len(insert_data))})
        ON CONFLICT(flight_id) DO UPDATE SET
        {', '.join([f"{col} = excluded.{col}" for col in new_columns])};
    '''
    print(insert_data)
    cursor.execute(insert_query, insert_data)
        
    conn.commit()
conn.close()

print("Done")

('ace175_1674698993_1674702094_KLAX_KSFO', 'A21N')
('a3b0fe_1675125526_1675128659_KLAX_KSFO', 'A319')
('a6cb48_1673061384_1673064350_KLAX_KSFO', 'A320')
('ad7ab8_1673183448_1673187207_KLAX_KSFO', 'A321')
('76cdb6_1673844821_1673968577_KLAX_KSFO', 'A359')
('06a12a_1672973656_1672977179_KLAX_KSFO', 'A35K')
('abf8e0_1673111431_1673115321_KLAX_KSFO', 'B38M')
('a2bba8_1673061037_1673064012_KLAX_KSFO', 'B39M')
('a620f5_1675118740_1675121856_KLAX_KSFO', 'B737')
('a0ad1d_1673290350_1673293730_KLAX_KSFO', 'B738')
('a46d4e_1674337182_1674340380_KLAX_KSFO', 'B739')
('71bc23_1672827572_1672830726_KLAX_KSFO', 'B744')
('71be09_1674719345_1674722480_KLAX_KSFO', 'B748')
('a8e24d_1675183500_1675186799_KLAX_KSFO', 'B752')
('aa8a26_1672849965_1672853902_KLAX_KSFO', 'B753')
('aa50a0_1673862801_1673866223_KLAX_KSFO', 'B763')
('aabd7f_1672916394_1672919997_KLAX_KSFO', 'B772')
('8990a0_1672820482_1672824479_KLAX_KSFO', 'B77L')
('a31a76_1674579223_1674582522_KLAX_KSFO', 'B789')
('a0566a_1672967624_1672970879_